In [1]:
import pandas as pd
import re
pd.options.display.max_colwidth = 400
pd.options.display.max_columns = None
pd.options.display.float_format = '{:.2f}'.format

In [2]:
trademe_raw = pd.read_json("./data/raw/trademe-2023-03-25.jsonl", lines=True)

In [3]:
#trademe_raw.head(2)

In [3]:
trademe_descriptions = trademe_raw.filter(['url','description'])
trademe_raw.drop('description', axis=1, inplace=True)

In [4]:
trademe_raw['year'] = trademe_raw.name.apply(lambda x : re.findall('(\d{4})', x))
trademe_raw['name'] = trademe_raw.name.apply(lambda x : re.sub(r"(\d{4})", "", x))
trademe_raw['vehicle_info_meta_all'] = trademe_raw.vehicle_info_meta + trademe_raw.vehicle_info_meta2

trademe_raw['vehicle_info_all'] = trademe_raw.apply( lambda x : {k: v for k, v in zip(tuple(x['vehicle_info_meta_all']), tuple(x['vehicle_info']))} , axis=1)

trademe_raw['ratings_fuel'] = trademe_raw.ratings_fuel.replace("L / 100Km",'', regex=True)
trademe_raw['ratings_carbon_emissions'] =  trademe_raw.ratings_carbon_emissions.replace(" Grams/Km",'', regex=True)

trademe_raw['background_item_check'].fillna("", inplace=True)
trademe_raw['background_value_check'].fillna("", inplace=True)
trademe_raw['price_details'].fillna("", inplace=True)

trademe_raw['background_check'] = trademe_raw.apply( lambda x : {k: v for k, v in zip(tuple(x['background_item_check']), tuple(x['background_value_check']))} , axis=1)
trademe_raw['price_details'] = trademe_raw.price_details.apply( lambda x : dict(map(lambda i: (x[i], x[i+1]), range(len(x)-1)[::2])) )


In [5]:
trademe_raw.drop(['vehicle_info_meta_all','vehicle_info_meta','vehicle_info_meta2','vehicle_info','background_item_check', 'background_value_check' ], axis=1, inplace=True)

In [6]:
price_details = pd.json_normalize(trademe_raw['price_details'])

background_check = pd.json_normalize(trademe_raw['background_check'])

vehicle_info_all = pd.json_normalize(trademe_raw['vehicle_info_all'])

trademe_processed = pd.concat([trademe_raw, price_details, background_check, vehicle_info_all], axis=1).drop(['price_details','background_check','vehicle_info_all'], axis=1)

trademe_processed.drop(["Number Plate:","Car Price","Clean Car Rebate/Fee","Year:",'Cost To Insure Rating:','Stereo Description:'], axis=1, inplace=True)

In [7]:
trademe_processed['views'].fillna(0.0,inplace=True)
trademe_processed['watchlist'].fillna(0.0,inplace=True)
trademe_processed['ratings_fuel'].fillna(0.0,inplace=True)

trademe_processed['price'] = trademe_processed['price'].str.replace(',', '').str.replace('.', '')
trademe_processed['price'].fillna(0,inplace=True)

trademe_processed['views'] = trademe_processed.views.astype(int)
trademe_processed['watchlist'] = trademe_processed.watchlist.astype(int)
trademe_processed['ratings_fuel'] = trademe_processed.ratings_fuel.astype(float)
trademe_processed['price'] = trademe_processed.price.astype(int)

trademe_processed['Kilometres'] = trademe_processed.Kilometres.str.replace(',', '').str.replace('Km', '').str.replace('More Than ','').astype(int)
trademe_processed['Engine Size'] = trademe_processed['Engine Size'].str.replace('cc', '')

trademe_processed.rename(columns={'Exterior Colour:': 'color'}, inplace=True)

def get_year(year):
    try:
        return year[0]
    except Exception:
        return 1700

trademe_processed['year'] = trademe_processed.year.apply(lambda x : get_year(x))
trademe_processed['year'] = trademe_processed.year.astype(int)


/var/folders/z4/164gc8sd291cdv48yt5hm2g40000gn/T/ipykernel_46711/2791229198.py:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  trademe_processed['price'] = trademe_processed['price'].str.replace(',', '').str.replace('.', '')


In [8]:
trademe_processed.head(3)

,url,name,price,model_type,period,location,views,ratings_fuel,ratings_carbon_emissions,vehicle_features,seller,datetime,watchlist,year,Clean Car Fee,On Road Costs,Total,Qualifies For Rebate,Price After Rebate Saving,Total (Excl. Rebate/Fee),Stolen Vehicle Check,Damaged Import Check,Re-Registered Check,Money Owing,Kilometres,Body Style,Seats,Fuel Type,Engine Size,Transmission,color,Doors:,Cylinders:,Number Of Owners:,Import History:,Registration Expires:,Wof Expires:
0,https://www.trademe.co.nz/a/motors/cars/nissan/juke/listing/4055504001?rsqid=f10b1c2d60604af280ef56155e3253d4-001,Nissan Juke,12974,Turbo,Listed Yesterday,"Mount Wellington, Auckland",245,8.30,192,"[Abs Brakes, Air Conditioning, Central Locking, Climate Control, Electric Windows, Spoiler, Turbo, Remote Locking]",Nz Cheap Cars,2023-03-25,0,2011,+$258.75,Excluded,"$12,974.00",NaN,NaN,NaN,Passed,Passed,Passed,Unavailable,125217,Rv/Suv,5,Petrol,1618Cc,Automatic,Blue,5 Door,NaN,NaN,NaN,NaN,NaN
1,https://www.trademe.co.nz/a/motors/cars/nissan/sunny/listing/4055626468?rsqid=f10b1c2d60604af280ef56155e3253d4-001,Nissan Sunny,0,NaN,NaN,"North Shore, Auckland",635,7.00,168,NaN,NaN,2023-03-25,37,2002,NaN,NaN,NaN,NaN,NaN,NaN,Passed,Passed,Passed,Passed,99554,Sedan,5,Petrol,1490Cc,Manual,Silver,4 Door,4 Cylinder,5+ Owners,Imported,Jan 2024,Mar 2024
2,https://www.trademe.co.nz/a/motors/cars/toyota/highlander/listing/4055791804?rsqid=f10b1c2d60604af280ef56155e3253d4-001,Toyota Highlander,33995,# 66 Blackout #,Listed Today,"Frankton, Hamilton",291,11.80,283,NaN,Auto 66,2023-03-25,12,2016,NaN,Included,"$33,995.00",NaN,NaN,NaN,Passed,Nz New,Passed,Unavailable,106742,Station Wagon,7,Petrol,3456Cc,Automatic,Blue,5 Door,NaN,NaN,4Wd,NaN,NaN


In [24]:
## name.str.contains('Outlander') or 
query = """( name.str.contains('Asx') or name.str.contains('Renegade') or name.str.contains('Compass') or name.str.contains('Ix35') or name.str.contains('X-Trail')  or name.str.contains('Sorento') or name.str.contains('Eclipse Cross') or name.str.contains('Rav4') or name.str.contains('Cx-3') or name.str.contains('Cx-5') or name.str.contains('Qashqai') or name.str.contains('Tucson') ) and location.str.contains('Auckland') and Transmission == 'Automatic' """

(
    trademe_processed.query(query)
    .filter(["name","year","price","model_type","Kilometres","color", "url"])
    .query("price > 0 and price <= 25000 and Kilometres < 70000 and year > 2017   ") #and color.isin(['Silver', 'Black','Grey','Ingot Silver,])
    .sort_values(by=["name","year","price"]).set_index(['name','year']) 
)

price                   model_type  Kilometres   color  \
name            year                                                           
 Jeep Renegade  2018  22000                      Limited       46100   White   
 Mitsubishi Asx 2018  21995                 Xls 2.0P/Cvt       64329     Red   
                2019  19990        Xls, Nz New, Low Km'S       64736   Black   
                2021  24950                  Ls 2.0P/Cvt       36000  Orange   
 Nissan Qashqai 2018  24990  St 2.0 Petrol Nz New On 20S       36704   Black   

                                                                                                                                       url  
name            year                                                                                                                        
 Jeep Renegade  2018   https://www.trademe.co.nz/a/motors/cars/jeep/renegade/listing/4050383779?rsqid=4d11466ccb704e4bb93091cbf99046d6-001  
 Mitsubishi Asx 2018  https://www.trademe.co.nz/a/motors/cars/mitsubishi/asx/listing/3992349691?rsqid=b8ca6f26127540e6a79cc441a0c3db2f-001  
                2019  https://www.trademe.co.nz/a/motors/cars/mitsubishi/asx/listing/4033718455?rsqid=4649fa74522f49039c95d51cea9d5fb2-001  
                2021  https://www.trademe.co.nz/a/motors/cars/mitsubishi/asx/listing/4051448860?rsqid=456ee0970e434b739d654ee7796129a6-001  
 Nissan Qashqai 2018  https://www.trademe.co.nz/a/motors/cars/nissan/qashqai/listing/4037401090?rsqid=509ddcbbc4d64f68aeb8fe5bb019c76f-001

In [13]:
#trademe_processed.query("name.str.contains('Trail') and location.str.contains('Auckland')")

In [15]:
# TODO: PROCESSING PERIOD
# TODO: VEHICLE FEATURES
# 
#  period
#  'Listed Today', 'Listed Within The Last 7 Days',
#        'Listed Yesterday', 'Listed Within The Last 30 Days',
#        'Listed More Than A Month Ago'


#trademe_processed.filter([ 'Registration Expires:']).value_counts()


# 'On Road Costs', 'Total',
#        'Qualifies For Rebate', 
#        'Price After Rebate Saving', 
#        'Clean Car Fee',
#        'Total (Excl. Rebate/Fee)',
#        'Stolen Vehicle Check',
#        'Damaged Import Check',
#        'Re-Registered Check', 
#        'Money Owing',
#        'Kilometres',
#        'Body Style',
#        'Seats', 'Fuel Type', 'Engine Size',
#        'Transmission', 'Exterior Colour:', 'Doors:',
#        'Cylinders:', 'Number Of Owners:', 'Import History:',
#        'Registration Expires:', 'Wof Expires:', 'Stereo Description:',
#        'Cost To Insure Rating:'

In [13]:
#trademe_processed.filter(['year','Year:']).head(50)

In [28]:
trademe_processed.name.nunique()

1411